# Verilog layer for the model 

What is the input bitwidth of model : Ie MNIST bitwidth. 

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
np.set_printoptions(linewidth=400)
import torch
import sys, os
from models import model_with_cfg, get_model_cfg
from utils.FC_wrapper import * 
from utils.matmul_optimization import read_index_array

In [5]:
n_neurons = 64
frac_bits = 20
# Downloads : PRETRAINED_URL: https://github.com/Xilinx/brevitas/releases/download/bnn_pynq-r1/tfc_1w1a-45185b4d.pth
cfg = get_model_cfg("TFC_1W1A")

model = fc_redefined(cfg)
# device = 'cuda:0'
device = 'cpu:0'

# package = torch.load('/home/ay/Desktop/masters/playground/matmul_optimization-master/jupyter/Amirali/model.tar', map_location='cuda:0')
# model_state_dict = package['state_dict']
# model.load_state_dict(model_state_dict, strict=True)
model = model.to(device)
model.eval()

FC_wrapper(
  (features): ModuleList(
    (0): InputAdjustment()
    (1): QuantIdentity(
      (input_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
      )
      (act_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
        (fused_activation_quant_proxy): FusedActivationQuantProxy(
          (activation_impl): Identity()
          (tensor_quant): ClampedBinaryQuant(
            (scaling_impl): ConstScaling(
              (restrict_clamp_scaling): _RestrictClampValue(
                (clamp_min_ste): Identity()
                (restrict_value_impl): FloatRestrictValue()
              )
              (value): StatelessBuffer()
            )
            (bit_width): BitWidthConst(
              (bit_width): StatelessBuffer()
            )
            (zero_point): StatelessBuffer()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
            (tensor_clamp_impl): TensorClamp(

In [6]:
# Print the weights 
for name, param in model.named_parameters():
    if param.requires_grad and 'weight' in name:
        print(f"{name}:\n{param.data}")

features.3.weight:
tensor([[ 0.3639, -0.6319, -0.6571,  ...,  0.9882, -0.9014,  0.9852],
        [-0.9971,  0.7938,  0.4874,  ...,  0.8853, -0.7393, -0.5158],
        [ 0.5060, -0.4909,  0.9286,  ...,  0.9236,  0.0393, -0.2485],
        ...,
        [-0.5924,  0.3219, -0.4118,  ..., -0.4066,  0.8367, -0.3115],
        [ 0.2590, -0.0870, -0.5748,  ..., -0.4640, -0.9647,  0.9518],
        [ 0.2091, -0.7778,  0.6700,  ...,  0.9951,  0.6560,  0.6186]])
features.4.weight:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
features.7.weight:
tensor([[ 0.4526, -0.7993, -0.3346,  ...,  0.7623, -0.9323, -0.6785],
        [ 0.5686, -0.5881, -0.7153,  ..., -0.4858, -0.8813, -0.2921],
        [ 0.3196,  0.8078,  0.3360,  ..., -0.6946,  0.3127,  0.7768],
        ..

In [4]:
# Assuming 'model' is already defined and loaded
for name, layer in model.named_modules():
        if isinstance(layer, nn.Linear):
            print(f"{name}: {layer}")

features.3: Linear(in_features=784, out_features=64, bias=False)
features.7: Linear(in_features=64, out_features=64, bias=False)
features.11: Linear(in_features=64, out_features=64, bias=False)
features.15: Linear(in_features=64, out_features=10, bias=False)


In [14]:
# Assuming 'model' is already defined and loaded
for name, layer in model.named_modules():
    # This loop iterates over all modules (layers) in the model. 
    # In the context of neural networks, a module can be any layer (like convolutional, linear, etc.).
    if isinstance(layer, nn.Linear):
        print(f"{name}: {layer}")
        W = layer.weight.cpu().detach().numpy() #.T   arrays Weights (Floats 32)
        weights = np.array(W * pow(2, 20),dtype=np.int32)
        print(weights.shape)

path = '/home/ay/Desktop/dinesh/matmul_optimization_github/jupyter/intermediate_files/v4_k=1_precision=20_latest'
array_fnames = ['{}/index_array-layer:{}-k:1-fbits:{}.npy'.format(path, i, frac_bits) for i in range(1,5)]
index_arrays = []
for fname in array_fnames:
    index_array = read_index_array(fname)
    layer_name = fname[117:124].replace(':','_')

    # index_ array contient deja N_rows et N_columns, et n'a pas beoins de n_bits_weights, mais on a besoin de n_bits_input pour la structure du module 
    generate_verilog_optimized(index_array , input_bitwidth = 8, filename = f'{layer_name}_')


features.3: Linear(in_features=784, out_features=64, bias=False)
(64, 784)
features.7: Linear(in_features=64, out_features=64, bias=False)
(64, 64)
features.11: Linear(in_features=64, out_features=64, bias=False)
(64, 64)
features.15: Linear(in_features=64, out_features=10, bias=False)
(10, 64)
Successfully generate verilog file!
Successfully generate verilog file!
Successfully generate verilog file!
Successfully generate verilog file!
